## 2 Limpieza de Repetidos

In [134]:
import pandas as pd
import json
import warnings
import numpy as np
from collections import defaultdict
import re
import ast

warnings.filterwarnings('ignore') 

In [135]:
df = pd.read_excel('../data/interm/Base 2025.xlsx')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4924 entries, 0 to 4923
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Evaluador        4924 non-null   object
 1   TipoAudifono     4924 non-null   object
 2   MarcaReferencia  4924 non-null   object
 3   Marca            4924 non-null   object
 4   Referencia       4924 non-null   object
 5   Frecuencia       4924 non-null   object
 6   Respuesta        4924 non-null   object
dtypes: object(7)
memory usage: 269.4+ KB


In [137]:
# Funcion para quitar parentesis
def limpiar_MarcaRef(texto):
    texto = texto.lower()
    texto = re.sub(r'\(.*?\)', '', texto)           # Elimina lo que está entre paréntesis
    texto = re.sub(r'[^a-z0-9\s]', '', texto)       # Conserva letras, números y espacios
    texto = re.sub(r'\s+', ' ', texto)              # Reemplaza múltiples espacios por uno solo
    return texto.strip()

def limpiar_Ref(texto):
    return re.sub(r'\s*\(.*?\)', '', texto).strip()

# funcion para convertir columna respuesta en una lista de numeros
def convertir_a_lista(respuesta):
    try:
        return ast.literal_eval(respuesta)
    except:
        return []


In [138]:
df['MarcaReferencia'] = df['MarcaReferencia'].astype(str)
df['Referencia'] = df['Referencia'].astype(str)
df['MarcaReferencia_Limpia'] = df['MarcaReferencia'].apply(limpiar_MarcaRef)
df['Referencia'] = df['Referencia'].apply(limpiar_Ref)
df['Respuesta'] = df['Respuesta'].apply(convertir_a_lista)


print(df[['MarcaReferencia', 'MarcaReferencia_Limpia']].head(10))

                     MarcaReferencia      MarcaReferencia_Limpia
0                  7Hz Salnotes Zero           7hz salnotes zero
1                       DUNU Titan S                dunu titan s
2                             EPZ Q5                      epz q5
3                     Etymotic ER2SE              etymotic er2se
4                  Final Audio E1000           final audio e1000
5                    Haylou Moripods             haylou moripods
6                         KZ ZSN Pro                  kz zsn pro
7         Moondrop Aria Snow Edition  moondrop aria snow edition
8  Simgot Audio EA500 (black nozzle)          simgot audio ea500
9                  Simgot Audio EM6L           simgot audio em6l


Separar el dataset en 3 datasets separados: 
- los que no tienen repetidos
- los que están repetidos dos veces
- Repetidos 3 o mas veces

In [141]:
conteo = df['MarcaReferencia_Limpia'].value_counts()

In [142]:
unicos = conteo[conteo == 1].index
duplicados_dos = conteo[conteo == 2].index
duplicados_tres_o_mas = conteo[conteo >= 3].index

In [143]:
df_unicos = df[df['MarcaReferencia_Limpia'].isin(unicos)]
df_dos = df[df['MarcaReferencia_Limpia'].isin(duplicados_dos)]
df_tres_o_mas = df[df['MarcaReferencia_Limpia'].isin(duplicados_tres_o_mas)]

#### 1. los de tres o mas repeticiones se escoge el mas cercano a la media

In [144]:
# Obtener el promedio de cada fila

df_tres_o_mas['Promedio_Respuesta'] = df_tres_o_mas['Respuesta'].apply(lambda x: sum(x)/len(x) if len(x) > 0 else None)

print(df_tres_o_mas[['MarcaReferencia_Limpia', 'Promedio_Respuesta']].head())


  MarcaReferencia_Limpia  Promedio_Respuesta
0      7hz salnotes zero            1.293741
1           dunu titan s            2.294216
2                 epz q5            3.364158
3         etymotic er2se           -0.415626
4      final audio e1000           -0.934590


In [145]:
# Agrupar por Referencia Limpia y traer el promedio de los promedios obtenidos
df_3_agrupado = df_tres_o_mas.groupby('MarcaReferencia_Limpia')['Promedio_Respuesta'].mean().reset_index()
df_3_agrupado.head()


,MarcaReferencia_Limpia,Promedio_Respuesta
0,64 audio a12t,1.712486
1,64 audio nio,2.769789
2,64 audio u12t,2.144385
3,64 audio u18t,3.439923
4,64 audio u4s,2.598405


In [146]:
# Llevar el promedio a la tabla y filtrar el mas cercano
df_tres_o_mas = df_tres_o_mas.merge(df_3_agrupado, on= 'MarcaReferencia_Limpia', how='left', suffixes=('','_Prom'))

print(df_tres_o_mas[['MarcaReferencia_Limpia', 'Promedio_Respuesta', 'Promedio_Respuesta_Prom']].head())

  MarcaReferencia_Limpia  Promedio_Respuesta  Promedio_Respuesta_Prom
0      7hz salnotes zero            1.293741                 1.924207
1           dunu titan s            2.294216                 1.944439
2                 epz q5            3.364158                 3.404672
3         etymotic er2se           -0.415626                -0.471879
4      final audio e1000           -0.934590                 1.445067


In [147]:
# Se trae el valor con el promedio mas cercano a el promedio general por referencia
df_tres_o_mas['Dif_respuestas'] = df_tres_o_mas['Promedio_Respuesta'] - df_tres_o_mas['Promedio_Respuesta_Prom']
df_tres_o_mas['Abs_Dif'] = df_tres_o_mas['Dif_respuestas'].abs()

In [148]:
df_cercanos = df_tres_o_mas.sort_values(by=['MarcaReferencia_Limpia', 'Abs_Dif'])
df_cercanos_final = df_cercanos.groupby('MarcaReferencia_Limpia').first().reset_index()

columnas_finales = ['Evaluador', 'TipoAudifono', 'MarcaReferencia', 'Marca', 'Referencia', 'Frecuencia', 'Respuesta']
df_tres_final = df_cercanos_final[columnas_finales]
df_unicos = df_unicos[columnas_finales]


In [149]:
df_tres_final.to_csv('3omas_final.csv')

In [150]:
# Consolidar el df resultante en el df_unicos
df_unicos_nuevo =pd.concat([df_unicos, df_tres_final], ignore_index=True)

#### 2. Obtenemos un ranking de los evaluadores mas constantes para usarlo de referencia en los que estan repetidos dos veces

In [151]:
# Contamos cuántas veces cada evaluador aparece en los datos ya validados
evaluador_frecuencia = df_unicos_nuevo['Evaluador'].value_counts().reset_index()
evaluador_frecuencia.columns = ['Evaluador', 'Conteo']
evaluador_frecuencia.head()

,Evaluador,Conteo
0,Rtings,535
1,Innerfidelity,526
2,oratory1990,282
3,Super Review,211
4,Headphone.com Legacy,140


In [ ]:
# se crea una tabla ranking de los evaluadores con mas reseñas dentro del archivo
ranking_evaluadores = {
    evaluador: i for i, evaluador in enumerate(evaluador_frecuencia['Evaluador'])
}

In [153]:
# Creamos una nueva columna con el ranking (número más bajo es más constante)
df_dos['Ranking_Evaluador'] = df_dos['Evaluador'].map(ranking_evaluadores)

# Si un evaluador no aparece en el ranking (es decir, no fue parte de df_validos), le damos un ranking alto
df_dos['Ranking_Evaluador'] = df_dos['Ranking_Evaluador'].fillna(len(ranking_evaluadores))


In [154]:
df_dos_nuevo = df_dos.sort_values(by=['MarcaReferencia_Limpia', 'Ranking_Evaluador'])
df_dos_nuevo = df_dos_nuevo.groupby('MarcaReferencia_Limpia').first().reset_index()

df_dos_nuevo = df_dos_nuevo[columnas_finales]

In [155]:
df_unicos_nuevo =pd.concat([df_unicos_nuevo, df_dos_nuevo], ignore_index=True)


In [156]:
df_unicos_nuevo['Referencia'] = df_unicos_nuevo.apply(
    lambda row: row['Referencia'].replace(row['Marca'], '').strip() 
    if str(row['Referencia']).startswith(str(row['Marca'])) else row['Referencia'], 
    axis=1
)

In [157]:
df_unicos_nuevo.to_csv('df_unicos_nuevo.csv')

In [ ]:
Marca_col = ['Marca']
Marcas = df_unicos_nuevo.groupby('Marca').first().reset_index()
Marcas = Marcas[Marca_col]

In [ ]:
Marcas.to_csv('Marcas.csv')